<a href="https://colab.research.google.com/github/ashu23481785/pyspark/blob/main/pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install findspark

import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType
from pyspark.sql.functions import col, lit, when, concat, date_format, to_date, unix_timestamp
from pyspark import SparkContext, SparkConf
conf = SparkConf().set("spark.ui.port", "4050")
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [ ]:
#--------------TO 10 Customer spent-----------------------
inputData = sc.textFile("/content/customerorders.csv")
mappedInput = inputData.map(lambda x: (x.split(",")[0], float(x.split(",")[2])))
aggData = mappedInput.reduceByKey(lambda x,y : x+y)
sortedData = aggData.sortBy(lambda x: x[1],False)
for customer, total in sortedData.take(10):
  print(f"Customer {customer}: {total:.2f}")

Customer 68: 6375.45
Customer 73: 6206.20
Customer 39: 6193.11
Customer 54: 6065.39
Customer 71: 5995.66
Customer 2: 5994.59
Customer 97: 5977.19
Customer 46: 5963.11
Customer 42: 5696.84
Customer 59: 5642.89


In [ ]:
#--------------AVG MOVIE RATING-----------------------
mvdata = sc.textFile("/content/moviedata.data")
spltData = mvdata.flatMap(lambda x: [(x.split("\t")[0],float(x.split("\t")[2]))])
#spltData.collect()
aggData = spltData.map(lambda x: (x[0],(x[1],1)))
aggData1=aggData.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
#aggData1.take(10)
avgRating = aggData1.mapValues(lambda x: round((x[0]/x[1]),1))
#avgRating.take(10)
sorData = avgRating.sortBy(lambda x: x[1],False)
sorData.take(10)

[('849', 4.9),
 ('688', 4.8),
 ('118', 4.7),
 ('507', 4.7),
 ('628', 4.7),
 ('928', 4.7),
 ('907', 4.6),
 ('686', 4.6),
 ('225', 4.5),
 ('242', 4.5)]

In [ ]:
#--------------RATING COUNT-----------------------
mvdata = sc.textFile("/content/moviedata.data")
spltData = mvdata.map(lambda x: x.split("\t")[2])
#spltData.take(10)
aggData = spltData.map(lambda x: (x,1))
#aggData.countByValue() -- use thi directly will give result as countbyvalues are action
#result.collect()
#aggData.take(10)
reducedData = aggData.reduceByKey(lambda x,y: x+y)
sortedData = reducedData.sortBy(lambda x: x[1],False)
sortedData.take(10)
#for rating,count in sortedData.collect():
 # print(f"{rating}")

[('4', 34174), ('3', 27145), ('5', 21201), ('2', 11370), ('1', 6110)]

In [ ]:
#--------------AVG No of linkedin connectiones for each age-----------------------
from math import *
frndsData = sc.textFile("/content/friendsdata.csv")
def parseline(line):
  fields = line.split(",")
  age = int(fields[2])
  numConn = int(fields[3])
  return (age,numConn)
spltRows = frndsData.map(parseline)
#spltRows.take(10)
mappedRdd = spltRows.map(lambda x:(int(x[0]),(int(x[1]),1)))
aggData = mappedRdd.reduceByKey(lambda x,y: (x[0]+y[0],x[1]+y[1]))
#aggData.take(10)
avgConn = aggData.mapValues(lambda x: floor(x[0]/x[1]))
sortData = avgConn.sortBy(lambda x: x[1],False)
sortData.take(10)

[(63, 384),
 (21, 350),
 (18, 343),
 (52, 340),
 (33, 325),
 (45, 309),
 (56, 306),
 (42, 303),
 (51, 302),
 (65, 298)]

In [ ]:
#Given an Input data set with name, age and city if age > 18
#add a new column that’s populated with ‘Y’ else ‘N
input_data = sc.textFile('/content/dataset1.text')
input_data.take(10)
def adultCheck(line):
  fields = line.split(",")
  name = fields[0]
  age = int(fields[1])
  city = fields[2]
  adult = 'Y' if age > 18 else 'N'
  return (name,age,city,adult)
mapped_date = input_data.map(adultCheck)
mapped_date.take(10)

[('sumit', 30, 'bangalore', 'Y'),
 ('kapil', 32, 'hyderabad', 'Y'),
 ('sathish', 16, 'chennai', 'N'),
 ('ravi', 39, 'bangalore', 'Y'),
 ('kavita', 12, 'hyderabad', 'N'),
 ('kavya', 19, 'mysore', 'Y')]

In [ ]:
#Given the input file where columns are stationId, timeOfTheReading, readingType,
#temperatureRecorded, and few other columns...
#We need to find the minimum temperature for each station id.
#Please do it using apache spark.

temp_data = sc.textFile("/content/tempdata.csv")
def parse_line(line):
  fields = line.split(',')
  stationId = fields[0]
  entryType = fields[2]
  temperature = int(fields[3])
  return (stationId, entryType, temperature)

mappedInp = temp_data.map(parse_line)
minTemp = mappedInp.filter(lambda x: x[1]== 'TMIN')
stationId = minTemp.map(lambda x: (x[0],x[2]))
#stationId.take(10)
aggData = stationId.reduceByKey(lambda x,y: min(x,y))
for station,temp in aggData.collect():
  print(f"{station} --> Min TMIN: {temp/10.0}°C")

ITE00100554 --> Min TMIN: -14.8°C
EZE00100082 --> Min TMIN: -13.5°C


In [ ]:
#********Total spent on search word Data*******#
campaign_data = sc.textFile("/content/bigdatacampaigndata.csv")
def parseline1(line):
  fields = line.split(",")
  searched_word = fields[0]
  total_cost = fields[10]
  if "data" in searched_word.lower():
    return("data",float(total_cost))
  else:
    return None
mapped_data = campaign_data.map(parseline1).filter(lambda x: x is not None)
aggData = mapped_data.reduceByKey(lambda x,y: x+y)
for word,cost in aggData.collect():
  print(f"total clicked charges for word '{word.upper()}': {cost:.2f}/-")

total clicked charges for word 'DATA': 17987.55/-


In [ ]:
#***********uses of broadcast variable***************************
campaign_data = sc.textFile("/content/bigdatacampaigndata.csv")
def boringWords():
  with open("/content/boringwords.text", "r") as file:
        lines = [line.strip() for line in file if line.strip()]
        boring_words = set(lines)
        return boring_words
#nameSet = sc.broadcast(boringWords)
boring_words_set = boringWords() # Execute the function on the driver
nameSet = sc.broadcast(boring_words_set) # Broadcast the resulting set
LoadedWrds = campaign_data.map(lambda x: (x.split(" ")[0],round(float(x.split(",")[10]),2)))
#LoadedWrds.take(10)
aggData = LoadedWrds.reduceByKey(lambda x,y: x+y).filter(lambda x: x[0]  not in nameSet.value).map(lambda x: (x[0],round(x[1],2)))
#aggData.take(10
aggData.collect()

[('big', 7990.32),
 ('hadoop', 2328.45),
 ('rdd', 19.98),
 ('leanr', 17.9),
 ('acadgild', 23.75),
 ('contentsbto', 24.66),
 ('bigdatauniversity,Broad', 24.48),
 ('intellipaat', 274.28),
 ('hive', 49.15),
 ('msc', 69.57),
 ('udacity', 69.73),
 ('4', 26.24),
 ('ivy', 64.72),
 ('iit', 82.64),
 ('scd', 24.82),
 ('birla', 29.83),
 ('deta', 49.54),
 ('learnbay', 29.81),
 ('eureka', 27.04),
 ('excelr', 24.92),
 ('intelipat,Broad', 24.25),
 ('cloudx', 52.34),
 ('datafalir,Broad', 29.78),
 ('bigdataphysicsclasses,Broad', 63.29),
 ('gcp', 22.96),
 ('cca', 62.61),
 ('edupristine', 30.0),
 ('prwatech,Broad', 24.76),
 ('https', 58.79),
 ('intellepat,Broad', 24.98),
 ('analytics', 88.59),
 ('prwatech', 24.06),
 ('trending', 21.78),
 ('simplilearn', 158.76),
 ('intellipaat,Broad', 1704.43),
 ('greatlearning', 24.05),
 ('nural', 34.22),
 ('qlikview', 29.96),
 ('cassandra', 19.81),
 ('edulab', 29.73),
 ('cloudxlab,Broad', 430.21),
 ('curso', 198.04),
 ('aws', 33.53),
 ('penerapan', 20.0),
 ('sumit', 0.

In [ ]:
#**********Example use of ACCUMULATOR********************
fileinput3 = sc.textFile("/content/samplefile.text")
#fileinput3.collect()
accum=sc.accumulator(0)
fileinput3.foreach(lambda x: accum.add(1) if x.strip()=="" else None)
print(f"Blank lines: {accum.value}")

Blank lines: 8


In [ ]:
from typing import List
#******READ FROM LIST AND COVERT LIST TO RDD***********
lst = ["WARN: Tuesday 4 September 0405",
           "ERROR: Tuesday 4 September 0408",
           "FATAL: Wednesday 5 September 1632",
           "ERROR: Friday 7 September 1854",
           "FATAL: Wednesday 5 September 1632",
            "ERROR: Tuesday 4 September 0408",
            "ERROR: Tuesday 4 September 0408"]
logs_rdd = sc.parallelize(lst)
print(logs_rdd.getNumPartitions()) #getnumpartition depends upon sc.defaultParallelism
print(logs_rdd.defaultParallelism)
mappedData = logs_rdd.map(lambda x: (x.split(":")[0],1))
TrnsfrmData = mappedData.reduceByKey(lambda x,y:x+y)
TrnsfrmData.take(10)

2


AttributeError: 'RDD' object has no attribute 'defaultParallelism'

In [ ]:
#Use of GROUPBYKEY vs REDUCEBYKEY
rf = sc.textFile('/content/bigLogNew.txt')
spltfl = rf.map(lambda x: x.split(':')[0])
prfl = spltfl.map(lambda x: (x,1))
#GROUPBYKEY#
#aggdt = prfl.groupByKey().mapValues(len) #took around 2-3 min for result
#display(aggdt.collect())

#REDUCEBYKEY#
aggdt = prfl.reduceByKey(lambda x,y:x+y)
aggdt.take(10)

[('WARN', 19995544), ('ERROR', 20004456)]

In [ ]:
#Amnt spent by each customer
inptdata = sc.textFile('/content/customerorders.csv')
def parseLine(line):
  fields = line.split(',')
  custId = fields[0]
  amountspent = float(fields[2])
  return (custId,amountspent)

mappedData = inptdata.map(parseLine)
aggData = mappedData.reduceByKey(lambda x,y: x+y)
filteredData = aggData.filter(lambda x: x[1]>=5000)
sortedData11 = filteredData.sortBy(lambda x: x[1],False) # Corrected index to 1 for amount
for customer,total in sortedData11.collect():
  print(f'Customer_id {customer} : total_spent {total:.2f}')

Customer_id 68 : total_spent 6375.45
Customer_id 73 : total_spent 6206.20
Customer_id 39 : total_spent 6193.11
Customer_id 54 : total_spent 6065.39
Customer_id 71 : total_spent 5995.66
Customer_id 2 : total_spent 5994.59
Customer_id 97 : total_spent 5977.19
Customer_id 46 : total_spent 5963.11
Customer_id 42 : total_spent 5696.84
Customer_id 59 : total_spent 5642.89
Customer_id 41 : total_spent 5637.62
Customer_id 0 : total_spent 5524.95
Customer_id 8 : total_spent 5517.24
Customer_id 85 : total_spent 5503.43
Customer_id 61 : total_spent 5497.48
Customer_id 32 : total_spent 5496.05
Customer_id 58 : total_spent 5437.73
Customer_id 63 : total_spent 5415.15
Customer_id 15 : total_spent 5413.51
Customer_id 6 : total_spent 5397.88
Customer_id 92 : total_spent 5379.28
Customer_id 43 : total_spent 5368.83
Customer_id 70 : total_spent 5368.25
Customer_id 72 : total_spent 5337.44
Customer_id 34 : total_spent 5330.80
Customer_id 9 : total_spent 5322.65
Customer_id 55 : total_spent 5298.09
Custom

# ***`PRACTICE SESSIONS BELOW`***

In [ ]:
#Big Data Campaign
inpfile=sc.textFile('/content/bigdatacampaigndata.csv')
splt_data = inpfile.map(lambda x: (float(x.split(",")[10]),x.split(",")[0].upper()))
splt_data.take(10)
flt_data = splt_data.flatMapValues(lambda x: x.split(" "))
#flt_data.take(10)
swp_data = flt_data.map(lambda x: (x[1],x[0]))
agg_data=swp_data.reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1],False).filter(lambda x: x[1]>=5000)
for word,amt in agg_data.collect():
  print(f'Most Searched keyword "{word}" : Total Amount Spent "{amt:.2f}"')

Most Searched keyword "DATA" : Total Amount Spent "16394.64"
Most Searched keyword "BIG" : Total Amount Spent "12889.28"
Most Searched keyword "IN" : Total Amount Spent "5774.84"


In [7]:
#Gen Auth Key as per below req
#want to generate an auth key in Spark (PySpark), with these rules:
#Total length = 10 bytes
#First 4 bytes → numbers (0–9)
#Next 4 bytes → special characters (like @#$%&*!)
#Last 2 bytes → alphabet characters (A–Z / a–z)

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import random
import string

def gen_auth_key():
  digits = ''.join(random.choices(string.digits,k=4))
  special_char=''.join(random.choices("!@#$%^&*()-_=+",k=4))
  chardata = ''.join(random.choices(string.ascii_letters,k=2))
  return digits + special_char + chardata

auth_key_udf = udf(gen_auth_key,StringType())

print(gen_auth_key())

7032!+&*Uz


In [20]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
from pyspark.sql.functions import *

data = [Row(1, "John", 30, "Sales", 50000.0),
 Row(2, "Alice", 28, "Marketing", 60000.0),
 Row(3, "Bob", 32, "Finance", 55000.0),
 Row(4, "Sarah", 29, "Sales", 52000.0),
 Row(5, "Mike", 31, "Finance", 58000.0)
]

schema = StructType([
    StructField("id",IntegerType(),nullable=False),
    StructField("name",StringType(),nullable=False),
    StructField("age",IntegerType(),nullable=False),
    StructField("dept",StringType(),nullable=False),
    StructField("salary",DoubleType(),nullable=False)
])

empDf = spark.createDataFrame(data,schema)
empDf.show()
empDf.createOrReplaceTempView("emp")
spark.sql("select dept,sum(salary) as total_salary from emp group by dept").show()

+---+-----+---+---------+-------+
| id| name|age|     dept| salary|
+---+-----+---+---------+-------+
|  1| John| 30|    Sales|50000.0|
|  2|Alice| 28|Marketing|60000.0|
|  3|  Bob| 32|  Finance|55000.0|
|  4|Sarah| 29|    Sales|52000.0|
|  5| Mike| 31|  Finance|58000.0|
+---+-----+---+---------+-------+

+---------+------------+
|     dept|total_salary|
+---------+------------+
|    Sales|    102000.0|
|Marketing|     60000.0|
|  Finance|    113000.0|
+---------+------------+

